In [261]:
import numpy as np 
import pandas as pd 
from datetime import date
from pandas_datareader import data
import requests 
import json
from dateutil.relativedelta import relativedelta


In [262]:
ratios = [0,0.236, 0.382, 0.5 , 0.618, 0.786,1,1.414,1.618]
fiblevels = np.array(ratios)

In [263]:
def RSI(series, period=14):
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    return 100 - 100 / (1 + rs)


# calculating Stoch RSI (gives the same values as TradingView)
# https://www.tradingview.com/wiki/Stochastic_RSI_(STOCH_RSI) 
def STRSIA(series, period=14, smoothK=3, smoothD=3):
    # Calculate RSI 
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    rsi = 100 - 100 / (1 + rs)

    # Calculate StochRSI 
    stochrsi  = (rsi - rsi.rolling(period).min()) / (rsi.rolling(period).max() - rsi.rolling(period).min())
    stochrsi_K = stochrsi.rolling(smoothK).mean()
    stochrsi_D = stochrsi_K.rolling(smoothD).mean()
    return stochrsi, stochrsi_K, stochrsi_D

In [264]:
ratios

[0, 0.236, 0.382, 0.5, 0.618, 0.786, 1, 1.414, 1.618]

In [265]:
## get the data 
import os
symbols= ['KEEPUSDT']
live = pd.to_datetime('today')
start = pd.to_datetime('2020-02-01')
end = pd.to_datetime('2021-10-21')

root_url = 'https://api.binance.com/api/v1/klines'
symbol = 'ADA'
interval = '1h'
url = root_url + '?symbol=' + symbol + '&interval=' + interval
print(url)


directory = 'BN-main'
table = []
   
        
        
        #now = df.close[df.close.idxmax()]
#get the live price 
        #array = filename.split("/")
        #symbol =array[0]+'USDT'
for symbol in symbols:
        url = root_url + '?symbol=' + symbol + '&interval=' + interval
        data = json.loads(requests.get(url).text)
        dflive = pd.DataFrame(data ,columns =['open_time',
              'o', 'h', 'l', 'c', 'v',
              'close_time', 'qav', 'num_trades',
              'taker_base_vol', 'taker_quote_vol', 'ignore'])
        dflive.open_time = pd.to_datetime(dflive['open_time'],unit='ms')
        dflive=dflive.loc[(dflive.open_time>start)]

dflive


https://api.binance.com/api/v1/klines?symbol=ADA&interval=1h


,open_time,o,h,l,c,v,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
0,2021-09-30 18:00:00,0.35810000,0.36730000,0.35800000,0.36330000,725760.00000000,1633028399999,263316.59570000,1130,348916.00000000,126617.09280000,0
1,2021-09-30 19:00:00,0.36330000,0.36470000,0.35750000,0.35770000,202395.00000000,1633031999999,73087.84140000,427,126309.00000000,45636.01560000,0
2,2021-09-30 20:00:00,0.35790000,0.35930000,0.35300000,0.35500000,290315.00000000,1633035599999,103481.58140000,631,194943.00000000,69484.44550000,0
3,2021-09-30 21:00:00,0.35480000,0.35930000,0.35470000,0.35930000,112776.00000000,1633039199999,40199.76720000,201,80664.00000000,28758.49230000,0
4,2021-09-30 22:00:00,0.35900000,0.36040000,0.35850000,0.35990000,84640.00000000,1633042799999,30398.74290000,166,33305.00000000,11973.53910000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
495,2021-10-21 09:00:00,0.78940000,0.80470000,0.78170000,0.79300000,3060552.00000000,1634810399999,2430286.61070000,6674,1683490.00000000,1338144.67930000,0
496,2021-10-21 10:00:00,0.79270000,0.79600000,0.78200000,0.78680000,1650056.00000000,1634813999999,1301798.53820000,2526,587693.00000000,464162.73140000,0
497,2021-10-21 11:00:00,0.78680000,0.78910000,0.74500000,0.75830000,3910095.00000000,1634817599999,2992431.23670000,7678,1808057.00000000,1382097.54250000,0
498,2021-10-21 12:00:00,0.75800000,0.76320000,0.74950000,0.75890000,2477691.00000000,1634821199999,1876693.76170000,4754,1153376.00000000,873834.32240000,0


In [266]:
ada15 = dflive[['open_time','o','h','l','c','v']]
ada15

,open_time,o,h,l,c,v
0,2021-09-30 18:00:00,0.35810000,0.36730000,0.35800000,0.36330000,725760.00000000
1,2021-09-30 19:00:00,0.36330000,0.36470000,0.35750000,0.35770000,202395.00000000
2,2021-09-30 20:00:00,0.35790000,0.35930000,0.35300000,0.35500000,290315.00000000
3,2021-09-30 21:00:00,0.35480000,0.35930000,0.35470000,0.35930000,112776.00000000
4,2021-09-30 22:00:00,0.35900000,0.36040000,0.35850000,0.35990000,84640.00000000
...,...,...,...,...,...,...
495,2021-10-21 09:00:00,0.78940000,0.80470000,0.78170000,0.79300000,3060552.00000000
496,2021-10-21 10:00:00,0.79270000,0.79600000,0.78200000,0.78680000,1650056.00000000
497,2021-10-21 11:00:00,0.78680000,0.78910000,0.74500000,0.75830000,3910095.00000000
498,2021-10-21 12:00:00,0.75800000,0.76320000,0.74950000,0.75890000,2477691.00000000


In [267]:
ada15.columns=['date','open','high','low','close','volume']
ada15

,date,open,high,low,close,volume
0,2021-09-30 18:00:00,0.35810000,0.36730000,0.35800000,0.36330000,725760.00000000
1,2021-09-30 19:00:00,0.36330000,0.36470000,0.35750000,0.35770000,202395.00000000
2,2021-09-30 20:00:00,0.35790000,0.35930000,0.35300000,0.35500000,290315.00000000
3,2021-09-30 21:00:00,0.35480000,0.35930000,0.35470000,0.35930000,112776.00000000
4,2021-09-30 22:00:00,0.35900000,0.36040000,0.35850000,0.35990000,84640.00000000
...,...,...,...,...,...,...
495,2021-10-21 09:00:00,0.78940000,0.80470000,0.78170000,0.79300000,3060552.00000000
496,2021-10-21 10:00:00,0.79270000,0.79600000,0.78200000,0.78680000,1650056.00000000
497,2021-10-21 11:00:00,0.78680000,0.78910000,0.74500000,0.75830000,3910095.00000000
498,2021-10-21 12:00:00,0.75800000,0.76320000,0.74950000,0.75890000,2477691.00000000


In [268]:
start = pd.to_datetime('2021-01-01')
ada15.date=pd.to_datetime(ada15['date'],unit='ms')
ada15= ada15.astype ({'open': float ,'high':float,'low':float ,'close':float ,'volume':float})


/Users/hjabbour/opt/anaconda3/envs/ML/lib/python3.7/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [269]:
start = pd.to_datetime('2021-01-01')
end = pd.to_datetime('2021-10-21')


ada2021 = ada15.loc[(ada15['date']> start)& (ada15['date']< end)].copy()
ada2021['RSI'] = RSI(ada2021['close'])
ada2021['RSI4']=RSI(ada2021['close'],56)
ada2021['RSI8']=RSI(ada2021['close'],112)
ada2021['STRSI'] = STRSIA(ada2021['close'])[0]
ada2021['STRSIK'] = STRSIA(ada2021['close'])[1]
ada2021['STRSID'] = STRSIA(ada2021['close'])[2]

ada2021['STRSI4'] = STRSIA(ada2021['close'],56,12,12)[0]
ada2021['STRSIK4'] = STRSIA(ada2021['close'],56,12,12)[1]
ada2021['STRSID4'] = STRSIA(ada2021['close'],56,12,12)[2]

ada2021['STRSI8'] = STRSIA(ada2021['close'],112,24,24)[0]
ada2021['STRSIK8'] = STRSIA(ada2021['close'],112,24,24)[1]
ada2021['STRSID8'] = STRSIA(ada2021['close'],112,24,24)[2]



print(ada2021.iloc[13:70,0:18])

                  date    open    high     low   close     volume        RSI  \
13 2021-10-01 07:00:00  0.3617  0.3669  0.3612  0.3665   305622.0        NaN   
14 2021-10-01 08:00:00  0.3669  0.3870  0.3664  0.3817  3009378.0  69.087137   
15 2021-10-01 09:00:00  0.3816  0.3838  0.3756  0.3799   912583.0  66.416079   
16 2021-10-01 10:00:00  0.3806  0.3901  0.3785  0.3846   607718.0  69.709195   
17 2021-10-01 11:00:00  0.3859  0.3920  0.3838  0.3908   601473.0  73.412815   
18 2021-10-01 12:00:00  0.3911  0.3941  0.3844  0.3884   421254.0  69.852395   
19 2021-10-01 13:00:00  0.3884  0.3915  0.3827  0.3827   556772.0  62.143797   
20 2021-10-01 14:00:00  0.3828  0.3873  0.3799  0.3835   516872.0  62.764876   
21 2021-10-01 15:00:00  0.3835  0.3915  0.3835  0.3897   660535.0  67.249391   
22 2021-10-01 16:00:00  0.3895  0.3937  0.3874  0.3904   404993.0  67.722062   
23 2021-10-01 17:00:00  0.3908  0.3936  0.3898  0.3922   268635.0  68.962530   
24 2021-10-01 18:00:00  0.3918  0.3940  

In [270]:
#maxlevels = [13,13,13,13,13,13,13]
#maxlevels = np.array(maxlevels)
#maxlevels

In [271]:
#minlevels = [11,11,11,11,11,11,11]
#minlevels = np.array(minlevels)
#minlevels

In [272]:
ada2021

,date,open,high,low,close,volume,RSI,RSI4,RSI8,STRSI,STRSIK,STRSID,STRSI4,STRSIK4,STRSID4,STRSI8,STRSIK8,STRSID8
0,2021-09-30 18:00:00,0.3581,0.3673,0.3580,0.3633,725760.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-09-30 19:00:00,0.3633,0.3647,0.3575,0.3577,202395.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-09-30 20:00:00,0.3579,0.3593,0.3530,0.3550,290315.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-09-30 21:00:00,0.3548,0.3593,0.3547,0.3593,112776.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-09-30 22:00:00,0.3590,0.3604,0.3585,0.3599,84640.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,2021-10-20 19:00:00,0.7876,0.7949,0.7845,0.7927,1396611.0,52.751987,50.309731,52.758970,1.000000,0.884757,0.851345,0.402843,0.226808,0.134411,0.069477,0.030712,0.077215
482,2021-10-20 20:00:00,0.7934,0.7948,0.7753,0.7817,1080023.0,46.993057,49.583988,52.394818,0.582377,0.832593,0.845963,0.283880,0.241876,0.149148,0.044922,0.032545,0.072411
483,2021-10-20 21:00:00,0.7817,0.7818,0.7745,0.7769,416446.0,44.699855,49.268218,52.236072,0.416080,0.666152,0.794501,0.241589,0.253852,0.164558,0.034218,0.033633,0.067959
484,2021-10-20 22:00:00,0.7766,0.7807,0.7701,0.7703,510432.0,41.687528,48.832836,52.017418,0.197634,0.398697,0.632481,0.175510,0.254731,0.179362,0.019474,0.034010,0.063791


In [273]:
high = ada2021.high
low = ada2021.low
maximums5 = high.rolling(5,5)
max5 = maximums5.max()
minimums5 = low.rolling(5,5)
min5 = minimums5.min()
maximums10 = high.rolling(10,10)
max10 = maximums10.max()
minimums10 = low.rolling(10,10)
min10 = minimums10.min()


In [274]:
len(max5)

486

In [275]:

ada2021['max5'] = max5
ada2021['max10'] = max10
ada2021['min5'] = min5
ada2021['min10'] = min10

ada2021.tail()

,date,open,high,low,close,volume,RSI,RSI4,RSI8,STRSI,...,STRSI4,STRSIK4,STRSID4,STRSI8,STRSIK8,STRSID8,max5,max10,min5,min10
481,2021-10-20 19:00:00,0.7876,0.7949,0.7845,0.7927,1396611.0,52.751987,50.309731,52.758970,1.000000,...,0.402843,0.226808,0.134411,0.069477,0.030712,0.077215,0.7970,0.8077,0.7720,0.7671
482,2021-10-20 20:00:00,0.7934,0.7948,0.7753,0.7817,1080023.0,46.993057,49.583988,52.394818,0.582377,...,0.283880,0.241876,0.149148,0.044922,0.032545,0.072411,0.7970,0.8077,0.7720,0.7681
483,2021-10-20 21:00:00,0.7817,0.7818,0.7745,0.7769,416446.0,44.699855,49.268218,52.236072,0.416080,...,0.241589,0.253852,0.164558,0.034218,0.033633,0.067959,0.7949,0.8077,0.7720,0.7714
484,2021-10-20 22:00:00,0.7766,0.7807,0.7701,0.7703,510432.0,41.687528,48.832836,52.017418,0.197634,...,0.175510,0.254731,0.179362,0.019474,0.034010,0.063791,0.7949,0.8077,0.7701,0.7701
485,2021-10-20 23:00:00,0.7705,0.7782,0.7687,0.7717,1225969.0,42.571607,48.930307,52.060367,0.079903,...,0.196293,0.259564,0.193901,0.022370,0.034724,0.059939,0.7949,0.8077,0.7687,0.7687


In [276]:
#maxnew = np.ones_like(ratios)
#maxnew

In [277]:
#addmin = mintest.tolist()
#addmax = maxtest.tolist()
#data = pd.DataFrame({"max":maxtest,"min":mintest},dtype='float')
#data = data.append(addmin,ignore_index=True)
#data1 = pd.DataFrame(maxtest)
#data2 = pd.DataFrame(mintest)
#data = pd.DataFrame(data1,data2)

In [278]:
#datamul = data.mul([2,1],axis='columns')
#datamul = data['max'].mul(ratios[7])

In [279]:
#datamul

In [280]:
#df = ada2021.loc[(ada2021['max10']>0)& (ada2021['min10']>0)]
df = ada2021
#len(df['date'])
average = df['max5']-df['min5']
avg = average.mean()
avg
df
#point = df.loc[(df['max5'].diff()>0)]
#pointm = df.loc[(df['min5'].diff()>0)]

df['support5']= df.min5.rolling(5,5).min()
df['support10']= df.min10.rolling(10,10).min()
df['resist5']= df.max5.rolling(5,5).max()
df['resist10']= df.max10.rolling(10,10).max()

## try 
#df['support5']= df.min5

#df['RSI'] = RSI(df['close'])
#df['STRSI4']=STRSIA(df['close'],56,12,12)
#df['STRSI8']=STRSIA(df['close'],112,24,24)


#point = df[(df['max5'] > df['max5'].shift(1)) &  (df['max5'].shift(1) > df['max5'].shift(2))]
#pointm = df[(df['min5'] < df['min5'].shift(1)) &  (df['min5'].shift(1) > df['min5'].shift(2))]
#point
#ada2021
#pointm

#for row in range(0,len(pointm)):
#    print(pointm['support5'])
df

,date,open,high,low,close,volume,RSI,RSI4,RSI8,STRSI,...,STRSIK8,STRSID8,max5,max10,min5,min10,support5,support10,resist5,resist10
0,2021-09-30 18:00:00,0.3581,0.3673,0.3580,0.3633,725760.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-09-30 19:00:00,0.3633,0.3647,0.3575,0.3577,202395.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-09-30 20:00:00,0.3579,0.3593,0.3530,0.3550,290315.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-09-30 21:00:00,0.3548,0.3593,0.3547,0.3593,112776.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-09-30 22:00:00,0.3590,0.3604,0.3585,0.3599,84640.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.3673,NaN,0.3530,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,2021-10-20 19:00:00,0.7876,0.7949,0.7845,0.7927,1396611.0,52.751987,50.309731,52.758970,1.000000,...,0.030712,0.077215,0.7970,0.8077,0.7720,0.7671,0.7681,0.7508,0.8077,0.8077
482,2021-10-20 20:00:00,0.7934,0.7948,0.7753,0.7817,1080023.0,46.993057,49.583988,52.394818,0.582377,...,0.032545,0.072411,0.7970,0.8077,0.7720,0.7681,0.7714,0.7508,0.8077,0.8077
483,2021-10-20 21:00:00,0.7817,0.7818,0.7745,0.7769,416446.0,44.699855,49.268218,52.236072,0.416080,...,0.033633,0.067959,0.7949,0.8077,0.7720,0.7714,0.7720,0.7508,0.8077,0.8077
484,2021-10-20 22:00:00,0.7766,0.7807,0.7701,0.7703,510432.0,41.687528,48.832836,52.017418,0.197634,...,0.034010,0.063791,0.7949,0.8077,0.7701,0.7701,0.7701,0.7527,0.8077,0.8077


In [281]:
buypoints = df[(df['STRSI'] < 0.01) &  (df['STRSI4'] < 0.01) & (df['STRSI8'] < 0.01) & (df['close'] <  df['support10'].shift(1)) & (df['RSI']<30)]
#buypoints = df[(df['STRSI'] < 0.01) &  (df['STRSI4'] < 0.01) & (df['STRSI8'] < 0.01)   & (df['RSI']<30)]

sellpoints = df[(df['STRSI'] == 1) &  (df['STRSI4'] == 1) & (df['STRSI8'] == 1) & (df['RSI']>90)]

#buypoints = df.loc[(df['STRSI'] <= 0.01) &  (df['STRSI4'] <= 0.01) & (df['STRSI8'] <= 0.01)]

sellpoints
buypoints

,date,open,high,low,close,volume,RSI,RSI4,RSI8,STRSI,...,STRSIK8,STRSID8,max5,max10,min5,min10,support5,support10,resist5,resist10
465,2021-10-20 03:00:00,0.7644,0.7682,0.7508,0.7532,925038.0,27.123247,47.852152,51.728617,0.0,...,0.09856,0.160641,0.785,0.8134,0.7508,0.7508,0.7508,0.7508,0.7852,0.8566


In [282]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px



#Declare figure


fig = go.Figure()

fig = make_subplots(rows=8, cols=1,subplot_titles=("Price","gap","empty", "RSI 1 4 8", "Stoch 1", "Stoch 4","Stoch 8","Stoch 1 4 8"))
    

    
# buy points     
fig.add_trace(
    go.Scatter(x=buypoints.date, y=buypoints['close'],name="buy", mode = "markers+text"), row=1, col=1 
)    

# sell points     
fig.add_trace(
    go.Scatter(x=sellpoints.date, y=sellpoints['close'],name="sell", mode = "markers"), row=1, col=1 
) 
    
# buy points     
fig.add_trace(
    go.Scatter(x=buypoints.date, y=buypoints['resist10'],
        name="resist10",
        text="resist10",
        textposition="top center"),
    row=1, col=1
)  
# resistance 10 


fig.add_trace(
    go.Scatter(x=df.date, y=df['RSI'],
        name="RSI",
        text="RSI",
        textposition="top center"),
    row=4, col=1
)

# support 10
fig.add_trace(
    go.Scatter(x=buypoints.date, y=buypoints['min10'],
        name="support10",
        text="support1010",
        textposition="top center"),
    row=1, col=1
)  

fig.add_trace(
    go.Scatter(x=df.date, y=df['RSI4'],name="RSI4"),
    row=4, col=1
)

fig.add_trace(
    go.Scatter(x=df.date, y=df['RSI8'],name="RSI8"),
    row=4, col=1
)

fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI'],name="STRSI"),
    row=5, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSIK'],name="STRSIK"),
    row=5, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSID'],name="STRSID"),
    row=5, col=1
)
    
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI4'],name="STRSI4"),
    row=6, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSIK4'],name="STRSIK4"),
    row=6, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSID4'],name="STRSID4"),
    row=6, col=1
)
    
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI8'],name="STRSI8"),
    row=7, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSIK8'],name="STRSIK8"),
    row=7, col=1
)

fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSID8'],name="STRSIK8"),
    row=7, col=1
)


fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI8'],name="STRSI8"),
    row=8, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI4'],name="STRSI4"),
    row=8, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI'],name="STRSI"),
    row=8, col=1
)


#add Traces

fig.add_trace(go.Candlestick(x=df.date,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'], name = 'market data'),row=1,col=1)

#fig.add_trace(go.Scatter(x=df['date'], y=df['max10']))

#fig.add_trace(go.Scatter(x=df['date'], y=df['min10']))

#fig.add_trace(go.Scatter(x=point['date'], y=point['resist5']))

#fig.add_trace(go.Scatter(x=pointm['date'], y=pointm['support5']))


#fig.add_shape(type="line",
#    x0=pointm['date'], y0=pointm['support5'], x1=max(pointm['date']), y1=pointm['support5'],
#    line=dict(
#        color="LightSeaGreen",
#        width=4,
#        dash="dashdot",
#    ))
fig.update_shapes(dict(xref='x', yref='y'))

#for index in range(0,len(pointm)):
    #fig.add_hline(y=pointm['support5'])

#fig.add_trace(go.Scatter(x=point['date'], y=point['resist10']))

#fig.add_trace(go.Scatter(x=pointm['date'], y=pointm['support10']))
#for i in range(len(df['date'])):
    #fig.add_hline(y=df[i]['min10'], line_dash="dot",
                  #annotation_text="support", 
                  #annotation_position="bottom right")

#fig.add_hline(y=df['max10'], line_dash="dot",
#              annotation_text="resitance", 
#              annotation_position="bottom right")



fig.update_layout(height=3000, width=800, title_text="top sub plots")
# X and Y Axis Update
fig.update_xaxes(
    title = 'Date',rangeslider_visible=False
)

#fig.update_yaxes(
#    title = 'Share Price (USD)'
#)

#Show
fig.show()